In [81]:
!pip install sqlalchemy==1.3.9

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [82]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [83]:
import csv, sqlite3

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [84]:
%sql sqlite:///my_data1.db

In [85]:
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

999

In [86]:
# Display the names of the unique launch sites  in the space mission

res = cur.execute("SELECT DISTINCT Launch_Site \
                   from SPACEXTBL")

res = res.fetchall()
res = [launch[0] for launch in res]

for i in res:
    if i is None:
        pass
    else:
        print(i)

CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


In [127]:
# Display 5 records where launch sites begin with the string 'CCA'
res = cur.execute("SELECT * \
                    FROM SPACEXTBL \
                    WHERE Launch_Site LIKE 'CCA%' \
                    LIMIT 5").fetchall()

for row in res:
    print(row)

('06/04/2010', '18:45:00', 'F9 v1.0  B0003', 'CCAFS LC-40', 'Dragon Spacecraft Qualification Unit', 0.0, 'LEO', 'SpaceX', 'Success', 'Failure (parachute)')
('12/08/2010', '15:43:00', 'F9 v1.0  B0004', 'CCAFS LC-40', 'Dragon demo flight C1, two CubeSats, barrel of Brouere cheese', 0.0, 'LEO (ISS)', 'NASA (COTS) NRO', 'Success', 'Failure (parachute)')
('22/05/2012', '7:44:00', 'F9 v1.0  B0005', 'CCAFS LC-40', 'Dragon demo flight C2', 525.0, 'LEO (ISS)', 'NASA (COTS)', 'Success', 'No attempt')
('10/08/2012', '0:35:00', 'F9 v1.0  B0006', 'CCAFS LC-40', 'SpaceX CRS-1', 500.0, 'LEO (ISS)', 'NASA (CRS)', 'Success', 'No attempt')
('03/01/2013', '15:10:00', 'F9 v1.0  B0007', 'CCAFS LC-40', 'SpaceX CRS-2', 677.0, 'LEO (ISS)', 'NASA (CRS)', 'Success', 'No attempt')


In [121]:
# Display the total payload mass carried by boosters launched by NASA (CRS)
res = cur.execute("SELECT sum(PAYLOAD_MASS__KG_) as 'Total Payload' \
                    FROM SPACEXTBL \
                    WHERE Customer LIKE 'NASA (CRS)'").fetchall()

print(res[0][0])

45596.0


In [120]:
# Display average payload mass carried by booster version F9 v1.1
res = cur.execute("SELECT avg(PAYLOAD_MASS__KG_) as 'Avarage Payload Mass'\
                    FROM SPACEXTBL \
                    WHERE Booster_Version LIKE 'F9 v1.1'").fetchall()

print(res[0][0])

2928.4


In [119]:
# List the date when the first succesful landing outcome in ground pad was acheived.

res = cur.execute("SELECT min(Date) \
                    FROM SPACEXTBL \
                    WHERE Landing_Outcome LIKE '%ground pad%'").fetchall()

print(res[0][0])

01/08/2018


In [123]:
# List the names of the boosters which have success in drone ship and have payload
# mass greater than 4000 but less than 6000
res = cur.execute("Select Booster_Version, Payload \
                    from SPACEXTBL \
                    where Mission_Outcome LIKE '%Success%' \
                    AND PAYLOAD_MASS__KG_ BETWEEN 4000 AND 6000").fetchall()
for i in res:
    print(res)

[('F9 v1.1', 'AsiaSat 8'), ('F9 v1.1 B1011', 'AsiaSat 6'), ('F9 v1.1 B1014', 'ABS-3A Eutelsat 115 West B'), ('F9 v1.1 B1016', 'Turkmen 52 / MonacoSAT'), ('F9 FT B1020', 'SES-9'), ('F9 FT B1022', 'JCSAT-14'), ('F9 FT B1026', 'JCSAT-16'), ('F9 FT B1030', 'EchoStar 23'), ('F9 FT  B1021.2', 'SES-10'), ('F9 FT B1032.1', 'NROL-76'), ('F9 B4 B1040.1', 'Boeing X-37B OTV-5'), ('F9 FT  B1031.2', 'SES-11 / EchoStar 105'), ('F9 B4 B1043.1', 'Zuma'), ('F9 FT  B1032.2', 'GovSat-1 / SES-16'), ('F9 B4  B1040.2', 'SES-12'), ('F9 B5 B1046.2', 'Merah Putih '), ('F9 B5 B1047.2', 'Es hail 2'), ('F9 B5 B1046.3', 'SSO-A '), ('F9 B5B1054', 'GPS III-01 '), ('F9 B5 B1048.3', 'Nusantara Satu, Beresheet Moon lander, S5'), ('F9 B5 B1051.2 ', 'RADARSAT Constellation, SpaceX CRS-18 '), ('F9 B5B1060.1', 'GPS III-03, ANASIS-II'), ('F9 B5 B1058.2 ', 'ANASIS-II, Starlink 9 v1.0'), ('F9 B5B1062.1', 'GPS III-04 , Crew-1')]
[('F9 v1.1', 'AsiaSat 8'), ('F9 v1.1 B1011', 'AsiaSat 6'), ('F9 v1.1 B1014', 'ABS-3A Eutelsat 115 We

In [130]:
# List the total number of successful and failure mission outcomes
res = cur.execute("Select count(Mission_Outcome) \
                    from SPACEXTBL \
                    where Mission_Outcome LIKE '%Success%'").fetchall()
print(res[0][0])

res = cur.execute("Select count(Mission_Outcome) \
                    from SPACEXTBL \
                    where Mission_Outcome NOT LIKE '%Success%'").fetchall()
print(res[0][0])



100
1


In [131]:
# List the names of the booster_versions which have carried the maximum payload mass.
res = cur.execute("Select Booster_Version \
                    from SPACEXTBL \
                    where (Select max(PAYLOAD_MASS__KG_) from SPACEXTBL)").fetchall()
print(res)

[('F9 v1.0  B0003',), ('F9 v1.0  B0004',), ('F9 v1.0  B0005',), ('F9 v1.0  B0006',), ('F9 v1.0  B0007',), ('F9 v1.1  B1003',), ('F9 v1.1',), ('F9 v1.1',), ('F9 v1.1',), ('F9 v1.1',), ('F9 v1.1',), ('F9 v1.1 B1011',), ('F9 v1.1 B1010',), ('F9 v1.1 B1012',), ('F9 v1.1 B1013',), ('F9 v1.1 B1014',), ('F9 v1.1 B1015',), ('F9 v1.1 B1016',), ('F9 v1.1 B1018',), ('F9 FT B1019',), ('F9 v1.1 B1017',), ('F9 FT B1020',), ('F9 FT B1021.1',), ('F9 FT B1022',), ('F9 FT B1023.1',), ('F9 FT B1024',), ('F9 FT B1025.1',), ('F9 FT B1026',), ('F9 FT B1029.1',), ('F9 FT B1031.1',), ('F9 FT B1030',), ('F9 FT  B1021.2',), ('F9 FT B1032.1',), ('F9 FT B1034',), ('F9 FT B1035.1',), ('F9 FT  B1029.2',), ('F9 FT B1036.1',), ('F9 FT B1037',), ('F9 B4 B1039.1',), ('F9 FT B1038.1',), ('F9 B4 B1040.1',), ('F9 B4 B1041.1',), ('F9 FT  B1031.2',), ('F9 B4 B1042.1',), ('F9 FT  B1035.2',), ('F9 FT  B1036.2',), ('F9 B4 B1043.1',), ('F9 FT  B1032.2',), ('F9 FT  B1038.2',), ('F9 B4 B1044',), ('F9 B4  B1041.2',), ('F9 B4  B103

In [152]:
# List the records which will display the month names, failure landing_outcomes
# in drone ship ,booster versions, launch_site for the months in year 2015.
res = cur.execute("SELECT CASE SUBSTR(Date, 4, 2) \
                    WHEN '01' THEN 'January' \
                    WHEN '02' THEN 'February' \
                    WHEN '03' THEN 'March' \
                    WHEN '04' THEN 'April' \
                    WHEN '05' THEN 'May' \
                    WHEN '06' THEN 'June' \
                    WHEN '07' THEN 'July' \
                    WHEN '08' THEN 'August' \
                    WHEN '09' THEN 'September' \
                    WHEN '10' THEN 'October' \
                    WHEN '11' THEN 'November' \
                    WHEN '12' THEN 'December' \
                    ELSE '' \
                    END AS Month, \
                    Landing_Outcome AS Failure_Landing_Outcomes, \
                    Booster_Version AS Booster_Version, \
                    Launch_Site AS Launch_Site \
                    FROM SPACEXTBL \
                    WHERE SUBSTR(Date, 7, 4) = '2015'").fetchall()

for i in res:
    print(i)

('October', 'Failure (drone ship)', 'F9 v1.1 B1012', 'CCAFS LC-40')
('November', 'Controlled (ocean)', 'F9 v1.1 B1013', 'CCAFS LC-40')
('February', 'No attempt', 'F9 v1.1 B1014', 'CCAFS LC-40')
('April', 'Failure (drone ship)', 'F9 v1.1 B1015', 'CCAFS LC-40')
('April', 'No attempt', 'F9 v1.1 B1016', 'CCAFS LC-40')
('June', 'Precluded (drone ship)', 'F9 v1.1 B1018', 'CCAFS LC-40')
('December', 'Success (ground pad)', 'F9 FT B1019', 'CCAFS LC-40')


In [161]:
# Rank the count of landing outcomes (such as Failure (drone ship) or Success
# (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.

res = cur.execute("SELECT Landing_Outcome, count(*) as Count \
                    FROM SPACEXTBL \
                    WHERE Date BETWEEN '04/06/2010' AND '20/03/2017' \
                    GROUP BY Landing_Outcome \
                    ORDER BY Count DESC").fetchall()
for r in res:
    print(res)

[('Success', 20), ('No attempt', 9), ('Success (drone ship)', 8), ('Success (ground pad)', 7), ('Failure (drone ship)', 3), ('Failure', 3), ('Failure (parachute)', 2), ('Controlled (ocean)', 2), ('No attempt ', 1)]
[('Success', 20), ('No attempt', 9), ('Success (drone ship)', 8), ('Success (ground pad)', 7), ('Failure (drone ship)', 3), ('Failure', 3), ('Failure (parachute)', 2), ('Controlled (ocean)', 2), ('No attempt ', 1)]
[('Success', 20), ('No attempt', 9), ('Success (drone ship)', 8), ('Success (ground pad)', 7), ('Failure (drone ship)', 3), ('Failure', 3), ('Failure (parachute)', 2), ('Controlled (ocean)', 2), ('No attempt ', 1)]
[('Success', 20), ('No attempt', 9), ('Success (drone ship)', 8), ('Success (ground pad)', 7), ('Failure (drone ship)', 3), ('Failure', 3), ('Failure (parachute)', 2), ('Controlled (ocean)', 2), ('No attempt ', 1)]
[('Success', 20), ('No attempt', 9), ('Success (drone ship)', 8), ('Success (ground pad)', 7), ('Failure (drone ship)', 3), ('Failure', 3), 